In [10]:
import torch
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler

data, y = torch.load('data/data.pkl')
train_data, test_data = data[:len(y)], data[len(y):]

np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(train_data, y, train_size=.8, )

In [11]:
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import numpy as np

std = StandardScaler()
lr = LogisticRegression()
adaboost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
# adaboost.set_params(**{'n_estimators': 398, 'learning_rate': 0.01, 'base_estimator__max_depth': 33,
#                        'base_estimator__min_samples_split': 164, 'base_estimator__min_samples_leaf': 20,
#                        'base_estimator__max_features': 9, 'base_estimator__max_leaf_nodes': 509}
#                     )
gdbt = GradientBoostingClassifier()
# gdbt.set_params(
#     **{'n_estimators': 399, 'learning_rate': 0.01, 'max_depth': 99, 'min_samples_split': 189, 'min_samples_leaf': 1,
#        'max_features': 9, 'max_leaf_nodes': 813}
# )
knn = KNeighborsClassifier()
# knn.set_params(**{'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
#                )
rf = RandomForestClassifier()
# rf.set_params(
#     **{'n_estimators': 201, 'max_depth': 99, 'min_samples_split': 2, 'min_samples_leaf': 1,
#        'max_features': 6, 'max_leaf_nodes': 57}
# )
svc = SVC(probability=True)
# svc.set_params(**{'C': 1.5, 'gamma': 0.16}
#                )
tree = DecisionTreeClassifier()
# tree.set_params(
#     **{'max_depth': 3, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 8}
# )

voting_model = StackingClassifier([('lr', lr), ('ada', adaboost), ('gdbt', gdbt),
                                   ('knn', knn), ('rf', rf), ('svc', svc), ('tree', tree)], n_jobs=-1,
                                  verbose=2,cv=10)
pipe = Pipeline([
    ('std', std),
    ('voting', voting_model)
])

pipe.fit(X_train, y_train)

Pipeline(steps=[('std', StandardScaler()),
                ('voting',
                 StackingClassifier(cv=10,
                                    estimators=[('lr', LogisticRegression()),
                                                ('ada',
                                                 AdaBoostClassifier(base_estimator=DecisionTreeClassifier())),
                                                ('gdbt',
                                                 GradientBoostingClassifier()),
                                                ('knn', KNeighborsClassifier()),
                                                ('rf',
                                                 RandomForestClassifier()),
                                                ('svc', SVC(probability=True)),
                                                ('tree',
                                                 DecisionTreeClassifier())],
                                    n_jobs=-1, verbose=2))])

In [12]:
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.8806179775280899, 0.8268156424581006)

In [13]:
pipe.fit(train_data, y)
pipe.score(train_data, y)

0.9079685746352413

In [14]:
import pandas as pd

best_model = pipe
y_test = best_model.predict(test_data.values)
result = pd.DataFrame(y_test, index=test_data.index, columns=['Survived'], )
result.to_csv('stacking.csv')

d:\project\python\.venv\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
